In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("CW2") \
    .getOrCreate()

sc = spark.sparkContext

In [2]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.sql import SparkSession
from preprocessing.Preprocessor import Preprocessor
from metrics.metrics import Metrics

In [3]:
pp = Preprocessor(spark)

In [4]:
train = pp.to_tfidf('Cleaned Data/train.csv', min_frequency=275., remove_stop_words = False)
train = train.sample(False, 0.6)
test = pp.to_tfidf('Cleaned Data/test.csv')

In [5]:
trainer = MultilayerPerceptronClassifier(layers=[1948, 64, 3], blockSize=32, seed=99)

In [6]:
%%time
model = trainer.fit(train)

CPU times: user 22.2 ms, sys: 10.7 ms, total: 32.9 ms
Wall time: 2min 26s


In [7]:
test_m = Metrics()
test_pred_and_labels = model.transform(test).select('prediction', 'label')
test_m.f1(test_pred_and_labels)
test_m.confusion_matrix(test_pred_and_labels)

F1 score for given DataFrame: 79.9

                Predictions
            negative    neutral     positive    
negative    2946        268         917         
neutral     330         565         584         
positive    717         405         9661        
(True labels are on the side)

